In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset
from tqdm import tqdm
import random
import os
from collections import defaultdict

In [3]:
device = 'cuda'

In [4]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
seed_everything(42)

# Dataset

In [6]:
df_train = pd.read_csv('./data/train_mean_median_max_min_current_next_prev.csv.zip')
df_test = pd.read_csv('./data/test_mean_median_max_min_current_next_prev.csv.zip')

In [7]:
df_train.head()

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X2_next_min,X4_next_min,X2_prev_min,X4_prev_min,X2_current_min,X4_current_min,to_next_game_1,to_next_game_2,to_prev_game_1,to_prev_game_2
0,0,2,908749,954,480270,935,71,79,53,89,...,780,905,867,931,954,935,40,1,2,7
1,1,1,438444,2162,486685,2154,32,39,48,49,...,2110,2096,2224,2126,2026,2034,17,13,8,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1822,2153,2092,2047,1883,2053,1,1,2,4
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1502,1416,1506,1564,1502,1474,1,1,1,26
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1263,1264,1239,1162,1202,1191,8,10,19,1


In [8]:
df_test['set'] = 'test'
df_train['set'] = 'train'

In [9]:
df_all = df_train.append(df_test)
df_all =df_all[df_train.columns]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [10]:
binarizer = MultiLabelBinarizer(classes=np.arange(0,101))

In [11]:
fig_1 = binarizer.fit_transform(df_all.iloc[:, 6:14].as_matrix())
fig_2 = binarizer.transform(df_all.iloc[:, 14:22].as_matrix())
columns_1 = [f'fig_1_{ind}' for ind in np.arange(0,101)]
columns_2 = [f'fig_2_{ind}' for ind in np.arange(0,101)]
fig_1 = pd.DataFrame(fig_1, columns=columns_1)
fig_2 = pd.DataFrame(fig_2, columns=columns_2 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [12]:
df_all = df_all.reset_index()
df_all = pd.concat([df_all, fig_1, fig_2], axis=1)

In [13]:
df_all = df_all.sort_values(by=['X21'])

In [14]:
df_all.head(20)['X21']

149649     7
2870902    7
1508534    7
2525939    7
3786209    7
2525745    7
2525125    7
4357924    7
3784646    7
3784630    7
1507189    7
1516805    7
1517551    7
1518739    7
1520450    7
3779162    7
528817     7
4358767    7
3778648    7
3778584    7
Name: X21, dtype: int64

In [15]:
player_to_exp = defaultdict(int)
exp_1 = []
exp_2 = []
for ind, line in tqdm(df_all.iterrows()):

      player_1 = line['X1']
      player_2 = line['X3']
      player_to_exp[player_1] += 1
      player_to_exp[player_2] += 1
      exp_1.append(player_to_exp[player_1])
      exp_2.append(player_to_exp[player_2])

4541257it [07:27, 10142.18it/s]


In [16]:
df_all['exp_1'] = exp_1
df_all['exp_2'] = exp_2

In [17]:
player_to_cum = defaultdict(int)
cum_1 = []
cum_2 = []
for ind, line in tqdm(df_all.iterrows()):

      current_1 = line.iloc[54:155].values
      current_2 = line.iloc[155:256].values

      player_to_cum[line['X1']] += current_1
      player_to_cum[line['X3']] += current_2
      cum_1.append(player_to_cum[line['X1']] * current_1)
      cum_2.append(player_to_cum[line['X3']] * current_2)

4541257it [19:33, 3871.01it/s]


In [18]:
df_all.head(20)['X21']

149649     7
2870902    7
1508534    7
2525939    7
3786209    7
2525745    7
2525125    7
4357924    7
3784646    7
3784630    7
1507189    7
1516805    7
1517551    7
1518739    7
1520450    7
3779162    7
528817     7
4358767    7
3778648    7
3778584    7
Name: X21, dtype: int64

In [19]:
columns_1_cum = [f'fig_1_{ind}_cum' for ind in np.arange(0,101)]
columns_2_cum = [f'fig_2_{ind}_cum' for ind in np.arange(0,101)]
fig_1_cum = pd.DataFrame(cum_1, columns=columns_1_cum)
fig_2_cum = pd.DataFrame(cum_2, columns=columns_2_cum)

In [20]:
df_all = df_all.reset_index()
df_all = pd.concat([df_all, fig_1_cum, fig_2_cum], axis=1)

In [21]:
df_test = df_all[df_all['set']=='test']
df_test = df_test.sort_values(by=['id'])

In [22]:
df_train = df_all[df_all['set']=='train']
df_train = df_train.sort_values(by=['id'])

In [23]:
df_train.head()

,level_0,index,id,X0,X1,X2,X3,X4,X5,X6,...,fig_2_91_cum,fig_2_92_cum,fig_2_93_cum,fig_2_94_cum,fig_2_95_cum,fig_2_96_cum,fig_2_97_cum,fig_2_98_cum,fig_2_99_cum,fig_2_100_cum
676977,0,0,0,2,908749,954,480270,935,71,79,...,0,0,0,0,0,0,0,0,0,43
2906432,1,1,1,1,438444,2162,486685,2154,32,39,...,33,0,0,28,0,0,0,0,0,0
2453996,2,2,2,1,596915,2066,711059,2081,43,21,...,0,0,0,0,0,0,0,0,0,0
2549512,3,3,3,1,625198,1508,66810,1474,7,1,...,0,0,0,0,0,0,5,0,0,0
4032063,4,4,4,2,228654,1202,542816,1196,53,35,...,0,0,0,0,49,0,0,0,0,0


In [24]:
fig_1_train = df_train[columns_1_cum].as_matrix()
fig_2_train = df_train[columns_2_cum].as_matrix()
fig_1_test = df_test[columns_1_cum].as_matrix()
fig_2_test = df_test[columns_2_cum].as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [25]:
fig_1_train = np.log(fig_1_train) + 1
fig_1_train[fig_1_train<0] = 0
fig_2_train = np.log(fig_2_train) + 1
fig_2_train[fig_2_train<0] = 0

fig_1_test = np.log(fig_1_test) + 1
fig_1_test[fig_1_test<0] = 0
fig_2_test = np.log(fig_2_train) + 1
fig_2_test[fig_2_test<0] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [26]:
df_train['1/to_next_game_1'] = df_train['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_next_game_2'] = df_train['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_1'] = df_train['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_train['1/to_prev_game_2'] = df_train['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

df_test['1/to_next_game_1'] = df_test['to_next_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_next_game_2'] = df_test['to_next_game_2'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_1'] = df_test['to_prev_game_1'].apply(lambda x: x if x==0 else 1/x)
df_test['1/to_prev_game_2'] = df_test['to_prev_game_2'].apply(lambda x: x if x==0 else 1/x)

In [27]:
rating_1_train = df_train[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1', 'exp_1']].values
rating_2_train = df_train[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2', 'exp_2']].values

rating_1_test =  df_test[['X2',
                           'X2_next_mean',  'X2_prev_mean','X2_current_mean',
                           'X2_next_median', 'X2_prev_median','X2_current_median',
                           'X2_next_max', 'X2_prev_max', 'X2_current_max', 
                           'X2_next_min', 'X2_prev_min', 'X2_current_min',
                           'to_next_game_1', 'to_prev_game_1', 
                           '1/to_next_game_1','1/to_prev_game_1', 'exp_1']].values
rating_2_test = df_test[['X4',
                           'X4_next_mean',  'X4_prev_mean','X4_current_mean',
                           'X4_next_median', 'X4_prev_median','X4_current_median',
                           'X4_next_max', 'X4_prev_max', 'X4_current_max', 
                           'X4_next_min', 'X4_prev_min', 'X4_current_min',
                           'to_next_game_2', 'to_prev_game_2', 
                           '1/to_next_game_2','1/to_prev_game_2', 'exp_2']].values

In [28]:
y_train = df_train['target']
rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train = shuffle(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42)

In [29]:
rating_1_train, rating_1_val, \
rating_2_train, rating_2_val, \
fig_1_train, fig_1_val, \
fig_2_train, fig_2_val, \
y_train, y_val = train_test_split(rating_1_train, rating_2_train, fig_1_train, fig_2_train, y_train, random_state=42, test_size=0.2)

In [30]:
# figs_train = np.concatenate([fig_1_train, fig_2_train], axis=1)
# figs_val = np.concatenate([fig_1_val, fig_2_val], axis=1)
# figs_test = np.concatenate([fig_1_test, fig_2_test], axis=1)

# rating_train = np.concatenate([rating_1_train, rating_2_train], axis=1)
# rating_val = np.concatenate([rating_1_val, rating_2_val], axis=1)
# rating_test = np.concatenate([rating_1_test, rating_2_test], axis=1)





# Data for torch

In [31]:
class TrainDataset(Dataset):
    def __init__(self, figs_1, figs_2, rating_1, rating_2, y):
        self.figs_1 = torch.tensor(figs_1, dtype=torch.float)
        self.figs_2 = torch.tensor(figs_2, dtype=torch.float)
        self.rating_1 = torch.tensor(rating_1, dtype=torch.float)
        self.rating_2 = torch.tensor(rating_2, dtype=torch.float)
        self.y = torch.tensor(y.tolist(), dtype=torch.float)
    
    def __len__(self):
        return len(self.figs_1)
    
    def __getitem__(self, idx):
        # можно добавить случайный своп
        return {
            'figs_1': self.figs_1[idx],
            'figs_2': self.figs_2[idx],
            'rating_1': self.rating_1[idx],
            'rating_2': self.rating_2[idx],
            'y': self.y[idx]
        }

class TestDataset(Dataset):
    def __init__(self,  figs_1, figs_2, rating_1, rating_2):
        self.figs_1 = torch.tensor(figs_1, dtype=torch.float)
        self.figs_2 = torch.tensor(figs_2, dtype=torch.float)
        self.rating_1 = torch.tensor(rating_1, dtype=torch.float)
        self.rating_2 = torch.tensor(rating_2, dtype=torch.float)
    
    def __len__(self):
        return len(self.figs_1)
    
    def __getitem__(self, idx):
        {
            'figs_1': self.figs_1[idx],
            'figs_2': self.figs_2[idx],
            'rating_1': self.rating_1[idx],
            'rating_2': self.rating_2[idx],
        }
        

In [32]:
def train_epoch(iterator, optimizer, criterion):
        model.train()
        epoch_loss = 0

        for i, batch in tqdm(enumerate(iterator)):
            targets = batch['y'].to(device)
            figs_1 = batch['figs_1'].to(device)
            figs_2 = batch['figs_2'].to(device)
            rating_1 = batch['rating_1'].to(device)
            rating_2 = batch['rating_2'].to(device)

            optimizer.zero_grad()

            logits = model(figs_1, figs_2, rating_1, rating_2).squeeze()
            loss = criterion(logits, targets)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def val_epoch(iterator,criterion):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                targets = batch['y'].to(device)
                figs_1 = batch['figs_1'].to(device)
                figs_2 = batch['figs_2'].to(device)
                rating_1 = batch['rating_1'].to(device)
                rating_2 = batch['rating_2'].to(device)
    
                logits = model(figs_1, figs_2, rating_1, rating_2).squeeze()
                loss = criterion(logits, targets)
                epoch_loss += loss.item()

        return epoch_loss / len(iterator)
    
def predict(iterator):
        model.eval()
        preds = []
        with torch.no_grad():
            for i, batch in tqdm(enumerate(iterator)):
                figs_1 = batch['figs_1'].to(device)
                figs_2 = batch['figs_2'].to(device)
                rating_1 = batch['rating_1'].to(device)
                rating_2 = batch['rating_2'].to(device)
                pred = model.predict(figs_1, figs_2, rating_1, rating_2)
                preds += pred.to('cpu').numpy().tolist()

        return preds

In [33]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1figs = nn.Linear(101, 128)
        self.bn1figs = nn.BatchNorm1d(128)        
        self.fc2figs = nn.Linear(128, 32)
        self.bn2figs = nn.BatchNorm1d(32)
       
        self.fc1rating = nn.Linear(18, 32)
        self.bn1rating = nn.BatchNorm1d(32)
        self.fc2rating = nn.Linear(32, 16)
        self.bn2rating = nn.BatchNorm1d(16)
        
        
        
        self.fc1res = nn.Linear(32*1 + 16*1, 32)
        self.bn1res = nn.BatchNorm1d(32)
        self.fc2res = nn.Linear(32, 1)
        
        self.do = nn.Dropout(0.5)
    
    def predict(self, figs_1, figs_2, rating_1, rating_2):
        result = self.forward(figs_1, figs_2, rating_1, rating_2)
        result = F.sigmoid(result)
        return result
        
    def forward(self, figs_1, figs_2, rating_1, rating_2):
        figs_1 = self.fc1figs(figs_1)
        figs_1 = self.bn1figs(figs_1)
        figs_1 = F.relu(figs_1)
        #figs_1 = self.do(figs_1)
        
        figs_1 = self.fc2figs(figs_1)
        figs_1 = self.bn2figs(figs_1)
        figs_1 = F.relu(figs_1)
        #figs_1 = self.do(figs_1)
        
        figs_2 = self.fc1figs(figs_2)
        figs_2 = self.bn1figs(figs_2)
        figs_2 = F.relu(figs_2)
        #figs_2 = self.do(figs_2)
        
        figs_2 = self.fc2figs(figs_2)
        figs_2 = self.bn2figs(figs_2)
        figs_2 = F.relu(figs_2)
        #figs_2 = self.do(figs_2)
        
        figs_diff = figs_1 - figs_2
        #figs_cross = torch.cross(figs_1, figs_2)
        
        
        rating_1 = self.fc1rating(rating_1)
        rating_1 = self.bn1rating(rating_1)
        rating_1 = F.relu(rating_1)
        #rating_1 = self.do(rating_1)
        
        rating_1 = self.fc2rating(rating_1)
        rating_1 = self.bn2rating(rating_1)
        rating_1 = F.relu(rating_1)
        #rating_1 = self.do(rating_1)
        
        
        rating_2 = self.fc1rating(rating_2)
        rating_2 = self.bn1rating(rating_2)
        rating_2 = F.relu(rating_2)
        #rating_2 = self.do(rating_2)
        
        rating_2 = self.fc2rating(rating_2)
        rating_2 = self.bn2rating(rating_2)
        rating_2 = F.relu(rating_2)
        #rating_2 = self.do(rating_2)
        
        rating_diff = rating_1 - rating_2
        #rating_cross = torch.cross(rating_1, rating_2)
        
        features = torch.cat((figs_diff, rating_diff), dim=-1)
        features = self.fc1res(features)
        features = self.bn1res(features)
        features = F.relu(features)
        #features = self.do(features)
        
        result = self.fc2res(features)
        return result

In [34]:
batch_size = 64
train_dataset = TrainDataset(fig_1_train, fig_2_train, rating_1_train, rating_2_train, y_train)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TrainDataset(fig_1_val, fig_2_val, rating_1_val, rating_2_val, y_val)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = TestDataset(fig_1_test, fig_2_test, rating_1_test, rating_2_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [35]:
rating_2_train.shape

(2906404, 18)

In [36]:
model = NN()
if device == 'cuda':
    model = model.cuda()

In [37]:
epochs = 40
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7, last_epoch=-1)

In [38]:
best_valid_loss = float('inf')
for epoch in range(1, epochs + 1):
    train_loss = round(train_epoch(train_dataloader, optimizer, criterion), 7)
    valid_loss = round(val_epoch(valid_dataloader, criterion), 7)
    if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), './data/models/same_weights_cum_log_and_exp.pt')

    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    scheduler.step()

45413it [05:05, 148.64it/s]
11354it [00:21, 526.18it/s]
1it [00:00,  5.03it/s]

Epoch: 1 
 Train Loss 0.5115197  Val loss 0.535146:


45413it [05:05, 148.69it/s]
11354it [00:21, 523.15it/s]
0it [00:00, ?it/s]

Epoch: 2 
 Train Loss 0.4980472  Val loss 0.5024408:


45413it [05:04, 149.12it/s]
11354it [00:21, 527.10it/s]
1it [00:00,  5.59it/s]

...

Epoch: 38 
 Train Loss 0.4719281  Val loss 0.4689717:


45413it [05:10, 146.23it/s]
11354it [00:21, 537.76it/s]
1it [00:00,  6.10it/s]

Epoch: 39 
 Train Loss 0.4719305  Val loss 0.4687099:


45413it [05:11, 145.81it/s]
11354it [00:21, 533.25it/s]

Epoch: 40 
 Train Loss 0.4718574  Val loss 0.4694913:


In [39]:
model.load_state_dict(torch.load('./data/models/same_weights_cum_log_and_exp.pt'))

<All keys matched successfully>

In [40]:
round(val_epoch(valid_dataloader, criterion), 7)

11354it [00:21, 534.57it/s]


0.4687099